In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from xgboost import XGBRegressor
import time
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()

# ============================================================
# 1. โหลดข้อมูล
# ============================================================
print("🔄 กำลังโหลดข้อมูล...")
df = pd.read_csv('experiment_design_27points.csv')

X = df[['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']].values
y = df.filter(like='Alpha_')

# เลือกแค่ 1-100 Hz
selected_hz = [f'Alpha_{i}Hz' for i in range(1, 101)]
y_selected = y[selected_hz].values

print(f"✅ Data: {len(df)} rows, {y_selected.shape[1]} outputs")

# Constraint
PROPORTION_SUM = 100
print(f"⚠️ Constraint: P1 + P2 = {PROPORTION_SUM}")

# ============================================================
# 2. เทรน XGBoost สำหรับ optimization (ใช้ค่าเฉลี่ย - เร็ว!)
# ============================================================
print("\n🤖 เทรน XGBoost สำหรับ optimization...")

y_mean = y_selected.mean(axis=1)
model_opt = XGBRegressor(
    n_estimators=100, max_depth=5, learning_rate=0.1,
    verbosity=0, random_state=42, tree_method='hist'
)
model_opt.fit(X, y_mean)
print(f"✅ เทรนเสร็จ!")

# ============================================================
# 3. เทรน XGBoost สำหรับแต่ละ Hz (สำหรับแสดงผล)
# ============================================================
print("\n🤖 เทรน XGBoost สำหรับแต่ละ Hz...")
models_hz = {}
for i, col in enumerate(selected_hz):
    m = XGBRegressor(
        n_estimators=50, max_depth=4, learning_rate=0.1,
        verbosity=0, random_state=42, tree_method='hist'
    )
    m.fit(X, y_selected[:, i])
    models_hz[col] = m
    if (i + 1) % 25 == 0:
        print(f"  ... {i + 1}/100")
print(f"✅ เทรนเสร็จ! ({time.time() - start_time:.1f}s)")

# ============================================================
# 4. ฟังก์ชันช่วย
# ============================================================
def expand_x(x3):
    return np.array([x3[0], PROPORTION_SUM - x3[0], x3[1], x3[2]])

def objective(x3):
    x4 = expand_x(x3).reshape(1, -1)
    return -model_opt.predict(x4)[0]  # maximize

def predict_all_hz(x4):
    """ทำนายค่า Alpha สำหรับทุก Hz"""
    x_arr = x4.reshape(1, -1)
    return np.array([models_hz[col].predict(x_arr)[0] for col in selected_hz])

# ============================================================
# 5. Bounds
# ============================================================
X_df = df[['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']]
p1_min = max(X_df['Proportion1'].min(), PROPORTION_SUM - X_df['Proportion2'].max())
p1_max = min(X_df['Proportion1'].max(), PROPORTION_SUM - X_df['Proportion2'].min())

bounds = [
    (p1_min, p1_max),
    (X_df['Temp_C'].min(), X_df['Temp_C'].max()),
    (X_df['Pressure_bar'].min(), X_df['Pressure_bar'].max()),
]

# ============================================================
# 6. Multi-start Optimization
# ============================================================
print("\n🔍 กำลังหาค่าสูงสุด...")

best_result = None
best_value = -np.inf

np.random.seed(42)
for i in range(20):
    x0 = [
        np.random.uniform(bounds[0][0], bounds[0][1]),
        np.random.uniform(bounds[1][0], bounds[1][1]),
        np.random.uniform(bounds[2][0], bounds[2][1]),
    ]
    result = minimize(objective, x0, method='L-BFGS-B', bounds=bounds)
    value = -result.fun
    if value > best_value:
        best_value = value
        best_result = result

# ============================================================
# 7. ผลลัพธ์
# ============================================================
optimal_x = expand_x(best_result.x)

print("\n" + "=" * 55)
print("🏆 ค่า Input ที่ให้ Alpha สูงสุด:")
print("=" * 55)
print(f"  Proportion1:   {optimal_x[0]:.4f}")
print(f"  Proportion2:   {optimal_x[1]:.4f}")
print(f"  ➡️ ผลรวม:       {optimal_x[0] + optimal_x[1]:.0f} ✅")
print(f"  Temp_C:        {optimal_x[2]:.4f}")
print(f"  Pressure_bar:  {optimal_x[3]:.4f}")
print("=" * 55)

# ============================================================
# 8. ทำนายค่า Alpha แต่ละ Hz
# ============================================================
predictions = predict_all_hz(optimal_x)

print(f"\n📊 ค่า Alpha ที่คาดการณ์สำหรับแต่ละ Hz:")
print("=" * 40)
print(f"{'Hz':>6} | {'Alpha':>10}")
print("-" * 20)

# แสดงทุกๆ 10 Hz
for i in range(0, 100, 10):
    hz = i + 1
    print(f"{hz:>6} | {predictions[i]:>10.4f}")

print("-" * 20)
print(f"{'Min':>6} | {predictions.min():>10.4f}")
print(f"{'Max':>6} | {predictions.max():>10.4f}")
print(f"{'Mean':>6} | {predictions.mean():>10.4f}")
print("=" * 40)

# แสดงทั้งหมด
print(f"\n📋 ค่า Alpha ทั้งหมด (1-100 Hz):")
print("-" * 60)
for i in range(0, 100, 5):
    line = ""
    for j in range(5):
        if i + j < 100:
            hz = i + j + 1
            line += f"{hz:>3}Hz:{predictions[i+j]:>.3f}  "
    print(line)

print(f"\n⏱️ Total time: {time.time() - start_time:.1f}s")
print("\n✅ เสร็จสิ้น!")

🔄 กำลังโหลดข้อมูล...
✅ Data: 27 rows, 100 outputs
⚠️ Constraint: P1 + P2 = 100

🤖 เทรน XGBoost สำหรับ optimization...
✅ เทรนเสร็จ!

🤖 เทรน XGBoost สำหรับแต่ละ Hz...
  ... 25/100
  ... 50/100
  ... 75/100
  ... 100/100
✅ เทรนเสร็จ! (1.1s)

🔍 กำลังหาค่าสูงสุด...

🏆 ค่า Input ที่ให้ Alpha สูงสุด:
  Proportion1:   46.1174
  Proportion2:   53.8826
  ➡️ ผลรวม:       100 ✅
  Temp_C:        127.7592
  Pressure_bar:  4.2117

📊 ค่า Alpha ที่คาดการณ์สำหรับแต่ละ Hz:
    Hz |      Alpha
--------------------
     1 |     0.3563
    11 |     0.6898
    21 |     0.8541
    31 |     0.5712
    41 |     0.3331
    51 |     0.7769
    61 |     0.9232
    71 |     0.3465
    81 |     0.5318
    91 |     0.2636
--------------------
   Min |     0.2469
   Max |     0.9232
  Mean |     0.5659

📋 ค่า Alpha ทั้งหมด (1-100 Hz):
------------------------------------------------------------
  1Hz:0.356    2Hz:0.575    3Hz:0.471    4Hz:0.623    5Hz:0.711  
  6Hz:0.428    7Hz:0.596    8Hz:0.662    9Hz:0.556   10Hz:0